In [ ]:
data = arrow::read_feather('/Users/sauterj1/Documents/Masters_Thesis/Figures_code/data/M1_Jake_Masters_thesis_cohort_analysis_matrix.feather')

In [ ]:
library(dplyr)
library(ggplot2)
library(magrittr)

In [ ]:
plot_umap <- function(points,
                      labels,
                      colors = NULL, 
                      alpha = 0.85, 
                      plot_labels = FALSE, 
                      plot_legend = FALSE, 
                      pt.cex = 0.005, 
                      legend.cex = 2.2, 
                      title = NA) {
  
  pad=0.2
  pch=19
  
  if (!is.factor(labels)) {
    new_levels = sort(unique(labels))
    labels = factor(labels, new_levels)
  }
  
  if (is.null(colors)) {
    require(RColorBrewer)
    n <- length(levels(labels))
    qual_col_pals = brewer.pal.info[brewer.pal.info$category == 'qual',]
    colors <- unlist(mapply(brewer.pal, qual_col_pals$maxcolors, rownames(qual_col_pals)))
    trans_colors <- scales::alpha(colors, alpha = alpha)
  } else {
    trans_colors <- colors
  }

  xylim = range(points)
  xylim = xylim + ((xylim[2]-xylim[1])*pad)*c(-0.5, 0.5)
  
  # ps is pointsize of text only, not symbols or points
  par(mar=c(0.7,0.7,5,0.7), ps=10)
  plot(xylim, xylim, type="n", axes=F, frame=F)
  rect(xylim[1], xylim[1], xylim[2], xylim[2], border="#aaaaaa", lwd=0.25)
  
  
  
  points(points[,1], points[,2], col=trans_colors[as.integer(labels)],
         cex=pt.cex, pch=pch)
  
  

  if (isTRUE(plot_labels)) {
    
    num_levels <- length(levels(labels))
    label_xs <- vector('numeric', num_levels)
    label_ys <- vector('numeric', num_levels)
  
    i <- 0
    for (label in levels(labels)) {
        i <- i + 1
        label = as.integer(label)
        label_xs[i] <- median(points[labels == label, 1])
        label_ys[i] <- median(points[labels == label, 2])
      }
  
    
    text(label_xs, 
         label_ys, 
         levels(labels), 
         cex = 2, 
         font = 1)
    }
  
  if (isTRUE(plot_legend)) {
    labels.u = levels(labels)
    legend.pos = "topleft"
    legend.text = as.character(labels.u)

    legend(legend.pos, legend=legend.text, inset=0.03,
           col=colors[seq_along(labels.u)],
           bty="n", pch=pch, cex = legend.cex, pt.cex = 3.2)
  }
  
  
  if (!is.na(title)) {
      title(main = list(title, cex = 5))
  }
}

## One cytometer colored per plot

In [ ]:
colnames(data)

In [ ]:
data %>% 
  group_by(anon_patient_id) %>% 
  sample_n(1) %>% 
  ungroup() %>% 
  dplyr::count(cytometer_used)

In [ ]:
umap_points <- 
  data %>% 
  select(UMAP1, UMAP2) %>% 
  as.data.frame()


cytomer
labels <- data$anon_patient_id
labels <- factor(labels)

require(RColorBrewer)
qual_col_pals = brewer.pal.info[brewer.pal.info$category == 'qual',]
my_colors <- unlist(mapply(brewer.pal, qual_col_pals$maxcolors, rownames(qual_col_pals)))

In [ ]:
umap_points <- 
  data %>% 
  select(UMAP1, UMAP2) %>% 
  as.data.frame()


cytometers <- data$cytometer_used
cytometers <- factor(cytometers)

require(RColorBrewer)
qual_col_pals = brewer.pal.info[brewer.pal.info$category == 'qual',]
my_colors <- unlist(mapply(brewer.pal, qual_col_pals$maxcolors, rownames(qual_col_pals)))

for (i in seq_along(levels(cytometers))) {

  cytometer <- levels(cytometers)[i]
  cytometer_name <- as.character(cytometer)
  
  print(cytometer_name)
  
  png(paste0('Umap_colored_by_patient_on_cytometer_', cytometer_name, '.png'), 
      width = 800*2, 
      height = 500*2)
      
  
  
  current_labels <- data$anon_patient_id
  current_labels <- ifelse(data$cytometer_used == cytometer_name, current_labels, 'Other')
  current_labels %<>% factor(levels = c(setdiff(current_labels, 'Other'), 'Other'))
  
  
  label_colors <- c(my_colors[1:nlevels(current_labels)-1], scales::alpha('gray', 0.1))
  
  plot_umap(umap_points, 
            labels = current_labels, 
            colors = label_colors, 
            alpha = 1, 
            plot_labels = FALSE, 
            plot_legend = TRUE, 
            pt.cex = 0.005, 
            legend.cex = 2.2, 
            title = cytometer_name)

  dev.off()

  print('Completed')
  
}

# Gathering them all

In [ ]:
library(png)
library(grid)
library(magrittr)

g1 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/Umap_colored_by_patient_on_cytometer_Canto 4.png') %>% 
  .[1:(nrow(.) - 50), 50:(ncol(.) - 50), ] %>%
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))

g2 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/Umap_colored_by_patient_on_cytometer_Canto 5.png') %>% 
  .[1:(nrow(.) - 50), 50:(ncol(.) - 50), ] %>%
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))


g3 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/Umap_colored_by_patient_on_cytometer_Canto 6.png') %>% 
  .[1:(nrow(.) - 50), 50:(ncol(.) - 50), ] %>%
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))

g4 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/Umap_colored_by_patient_on_cytometer_Fortessa 1.png') %>% 
  .[1:(nrow(.) - 50), 50:(ncol(.) - 50), ] %>%
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))

g5 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/Umap_colored_by_patient_on_cytometer_Fortessa 2.png') %>% 
  .[1:(nrow(.) - 50), 50:(ncol(.) - 50), ] %>%
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))

g6 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/Umap_colored_by_patient_on_cytometer_Symphony 2.png') %>% 
  .[1:(nrow(.) - 50), 50:(ncol(.) - 50), ] %>%
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))

g7 <- 
  readPNG('~/Documents/Masters_Thesis/Figures_code/Umap_colored_by_patient_on_cytometer_Symphony 3.png') %>% 
  .[1:(nrow(.) - 50), 50:(ncol(.) - 50), ] %>%
  rasterGrob(interpolate=TRUE, 
             width=ggplot2::unit(1,"npc"),
             height=ggplot2::unit(1,"npc"))



p1 <- ggplot2::ggplot() + ggplot2::annotation_custom(g1, -Inf, Inf, -Inf, Inf)
p2 <- ggplot2::ggplot() + ggplot2::annotation_custom(g2, -Inf, Inf, -Inf, Inf)
p3 <- ggplot2::ggplot() + ggplot2::annotation_custom(g3, -Inf, Inf, -Inf, Inf)
p4 <- ggplot2::ggplot() + ggplot2::annotation_custom(g4, -Inf, Inf, -Inf, Inf)
p5 <- ggplot2::ggplot() + ggplot2::annotation_custom(g5, -Inf, Inf, -Inf, Inf)
p6 <- ggplot2::ggplot() + ggplot2::annotation_custom(g6, -Inf, Inf, -Inf, Inf)
p7 <- ggplot2::ggplot() + ggplot2::annotation_custom(g7, -Inf, Inf, -Inf, Inf)


library(patchwork)

theme_border <- theme_light() + 
    theme(plot.background = element_rect(fill = 'white', colour = 'black', size = 1.75))

p <- 
  list(p1, p2, p3, p4, p5, p6, p7) %>%
  patchwork::wrap_plots(ncol = 3) + 
  plot_annotation(caption = '')

ggsave('Patchwork_Single_Cytometer_UMAPs.png', plot = p, 
       height = 12, width = 17)

print('Done')